In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import re
import psycopg2

## Fetching data

In [2]:
# Connect to database
conn = psycopg2.connect(
    host='covid19db.org',
    port=5432,
    dbname='covid19',
    user='covid19',
    password='covid19')
cur = conn.cursor()

In [3]:
# Fetch data using SQL Query
sql_command = """SELECT * FROM world_bank"""
df_wb = pd.read_sql(sql_command, conn)

sql_command = """SELECT * FROM surveys"""
df_surveys = pd.read_sql(sql_command, conn)

## World Bank Table

In [5]:
# To group by country and indicator_name (survey question) and look at values
df_wb.groupby(["country", "indicator_name"])\
    [["country", "indicator_name", "value", "year"]].mean("value")

value  \
country     indicator_name                                                   
Afghanistan ARI treatment (% of children under 5 taken to a...   61.500000   
            Access to clean fuels and technologies for cook...   32.440000   
            Access to electricity (% of population)              98.713203   
            Access to electricity, rural (% of rural popula...   98.272872   
            Access to electricity, urban (% of urban popula...  100.000000   
...                                                                    ...   
Zimbabwe    Women who believe a husband is justified in bea...   21.400000   
            Women who believe a husband is justified in bea...   14.500000   
            Women who were first married by age 15 (% of wo...    3.700000   
            Women who were first married by age 18 (% of wo...   32.400000   
            Women's share of population ages 15+ living wit...   59.800000   

                                                                year  
country     indicator_name                                            
Afghanistan ARI treatment (% of children under 5 taken to a...  2015  
            Access to clean fuels and technologies for cook...  2016  
            Access to electricity (% of population)             2018  
            Access to electricity, rural (% of rural popula...  2018  
            Access to electricity, urban (% of urban popula...  2018  
...                                                              ...  
Zimbabwe    Women who believe a husband is justified in bea...  2015  
            Women who believe a husband is justified in bea...  2015  
            Women who were first married by age 15 (% of wo...  2015  
            Women who were first married by age 18 (% of wo...  2015  
            Women's share of population ages 15+ living wit...  2018  

[307665 rows x 2 columns]

In [6]:
# Just looking at survey results from 2019
df_wb_2019 = df_wb[df_wb.year==2018].dropna(subset=["value"])[
    ["country", "indicator_name", "value"]]

In [7]:
# Showing the top questions that have been asked by the most number of countries

df_wb_2019.groupby("indicator_name").count()\
           .sort_values(by="value", ascending=False).head(10)

,country,value
indicator_name,,
"Mammal species, threatened",215,215
"Bird species, threatened",215,215
"Plant species (higher), threatened",215,215
"Fish species, threatened",215,215
Adjusted savings: mineral depletion (current US$),214,214
Access to electricity (% of population),214,214
"Access to electricity, urban (% of urban population)",212,212
Terrestrial protected areas (% of total land area),211,211
Terrestrial and marine protected areas (% of total territorial area),210,210


In [8]:
# Showing the top questions that have been asked by the most number of countries
# that include certain keywords 

df_wb_2019[df_wb_2019.indicator_name.str\
           .contains("education")]\
           .groupby("indicator_name").count()\
           .sort_values(by="value", ascending=False).head(10)

,country,value
indicator_name,,
Adjusted savings: education expenditure (% of GNI),198,198
"Compulsory education, duration (years)",178,178
Adjusted savings: education expenditure (current US$),178,178
"Preprimary education, duration (years)",177,177
"Primary education, pupils (% female)",89,89
"Primary education, teachers",89,89
"Primary education, teachers (% female)",89,89
"Primary education, pupils",89,89
"Secondary education, general pupils",86,86


## Survey Table

Currently, the survey properties are in one long JSON. We can explode each property in the JSON into its own line so that each survey question-answer pair gets its own line

In [9]:
df_surveys.head(2)

,source,wave,gid,country,countrycode,adm_area_1,adm_area_2,adm_area_3,samplesize,properties
0,EVS,1981-1984,[BEL],Belgium,BEL,None,None,None,1145,{'A001': {'Label': 'Important in life: Family'...
1,EVS,1981-1984,[CAN],Canada,CAN,None,None,None,1254,{'A001': {'Label': 'Important in life: Family'...


In [10]:
def explode_survey_props(props):
    q_label, label, category, frequency = [], [], [], []

    for line in props:
        if "X023" in line: continue # this one too complicated to clean
        if "X051" in line: continue # ethnic group; this one has no categories
        if "original_region_code" in line: break #this one is not related
            
        for freq in props[line]['Frequencies']:      
            if re.search("[A-Z]\d+_\d+_(-*\d+)", freq):
                q_val = re.search("[A-Z]\d+_\d+_(-*\d+)", freq).group(1)
            else:
                q_val = re.search("[A-Z]\d\d\d[A-Z]*_(-*\d+)", freq).group(1)
            q_label.append(line)
            label.append(props[line]['Label'])
            frequency.append(props[line]['Frequencies'][freq])        

            if "X002" in freq or "X003" in freq: #special case of birth year/age
                category.append(freq)
            else:
                category.append(props[line]['Categories'][q_val])

    return q_label, label, category, frequency

In [11]:
# Build up in arrays
waves, gids, countries, samplesizes = [],[],[],[]
q_labels, labels, categories, frequencies = [],[],[],[]

# Iterate through the survey to explode each json
for i in range(len(df_surveys)):
    props = df_surveys.properties[i]
    q_label, label, category, frequency = explode_survey_props(props)
    
    # These are copied over from the original dataframe
    source = [df_surveys.source[i]] * len(q_label)
    wave = [df_surveys.wave[i]] * len(q_label)
    gid = [df_surveys.gid[i]] * len(q_label)
    country = [df_surveys.country[i]] * len(q_label)
    samplesize = [df_surveys.samplesize[i]] * len(q_label)

    waves.extend(wave)
    gids.extend(gid)
    countries.extend(country)
    samplesizes.extend(samplesize)
    q_labels.extend(q_label)
    labels.extend(label)
    categories.extend(category)
    frequencies.extend(frequency)

In [12]:
# Build the dataframe... this takes some time because the arrays are long

cols = ['wave', 'gid', 'country', 'samplesize', 
        'q_label', 'label', 'category', 'frequency']
survey_explode_df = pd.DataFrame([waves, gids, countries, samplesizes, 
                                  q_labels, labels, categories, frequencies]).T
survey_explode_df.columns=cols
survey_explode_df.frequency = survey_explode_df.frequency.apply(float)

In [13]:
print(len(survey_explode_df))
display(survey_explode_df.sample(5))

833664


,wave,gid,country,samplesize,q_label,label,category,frequency
661109,2010-2014,[ESP.2_1],Spain,33,X028,Employment status,Self employed,0.121212
272821,2005-2009,[MAR],Morocco,1200,E037,Government responsibility,4,0.033333
103880,2008-2010,[NOR],Norway,1090,X003,Age,X003_23,0.019688
790382,2008-2010,"[PRT.3_1, PRT.8_1, PRT.12.4_1, PRT.14_1, PRT.1...",Portugal,183,X036,Profession/job,"""Missing; Unkown""",0.000000
226109,1999-2004,[PRI],Puerto Rico,720,X003,Age,X003_-1,0.000000


In [14]:
# Showing 20 of the unique questions asked in the survey
survey_questions = survey_explode_df.label.unique()
print(len(survey_questions))
print(survey_questions[:20])

74
['Important in life: Family' 'Important in life: Friends'
 'Important in life: Work' 'State of health (subjective)'
 'Important child qualities: independence'
 'Important child qualities: feeling of responsibility'
 'Important child qualities: obedience'
 'Member: Belong to religious organization'
 'Member: Belong to sports or recreation'
 'Active/Inactive membership of church or religious organization'
 'Active/Inactive membership of sport or recreation'
 'Most people can be trusted' 'Satisfaction with your life'
 'How much freedom of choice and control'
 'Schwartz: It is important to this person living in secure surroundings'
 'Schwartz: It is important to this person to have a good time'
 'Schwartz: It is important to this person to always behave properly'
 'Schwartz: It is important to this person to do something for the good of society'
 'Social position: People in their 20s'
 'Social position: People in their 40s']


In [15]:
# The unique waves of years
survey_explode_df.wave.unique()

array(['1981-1984', '1990-1993', '1999-2001', '2008-2010', '1989-1993',
       '1994-1998', '1999-2004', '2005-2009', '2010-2014'], dtype=object)

In [16]:
# How do countries respond to Family Values in 2010-2014?
mask = (survey_explode_df.wave=='2010-2014')\
    &(survey_explode_df.label=="Important in life: Family")
survey_explode_df[mask].groupby(["country", "category"]).sum().head(20)

frequency
country   category                       
Algeria   Don´t know             0.003333
          Missing; Unknown       0.000000
          No answer              0.000833
          Not asked in survey    0.000000
          Not at all important   0.011667
          Not very important     0.011667
          Rather important       0.041667
          Very important         0.930833
Argentina Don´t know             0.000000
          Missing; Unknown       0.000000
          No answer              0.002467
          Not asked in survey    0.000000
          Not at all important   0.002577
          Not very important     0.004394
          Rather important       0.101518
          Very important         0.889044
Armenia   Don´t know             0.001592
          Missing; Unknown       0.002474
          No answer              0.000573
          Not asked in survey    0.000000

In [17]:
# How do countries respond to survey questions that include the word "important"?
survey_explode_df[survey_explode_df.label.str.contains("Important|important")]\
    .groupby("label").agg({"country": "nunique", "wave":"nunique"})

,country,wave
label,,
Important child qualities: feeling of responsibility,107,9
Important child qualities: independence,107,9
Important child qualities: obedience,107,9
Important in life: Family,107,9
Important in life: Friends,107,9
Important in life: Work,107,9
It is not important for me to know about science in my daily life,107,9
Most important: first choice,107,9
Most important: second choice,107,9


In [18]:
# Group survey questions by country, wave, label, category, then sum
survey_explode_df.groupby(["country", "wave", "label", "category"])\
    [["samplesize", "frequency"]].sum()

frequency
country  wave      label                                              category                   
Albania  1994-1998 Active/Inactive membership of church or religio... Active member      0.048048
                                                                      Don´t know         0.000000
                                                                      Inactive member    0.162162
                                                                      Missing; Unknown   0.000000
                                                                      No answer          0.003003
...                                                                                           ...
Zimbabwe 2010-2014 Year of birth                                      X002_1994          0.006580
                                                                      X002_1995          0.000000
                                                                      X002_1996          0.000000
                                                                      X002_1997          0.000000
                                                                      X002_1999          0.000000

[354576 rows x 1 columns]